In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
import json
import os
from copy import deepcopy
from pathlib import Path
from typing import Any

import blobfile as bf
import numpy as np
import tqdm
import plotly.express as px
import plotly.graph_objects as go
import torch
from cupbearer import data, detectors, models, scripts, tasks, utils
from IPython.display import Markdown, display
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from transformer_lens import HookedTransformer
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
)

# Setup

In [3]:
os.environ["NO_GCE_CHECK"] = "true"

In [4]:
class ARCDataset(torch.utils.data.Dataset):
    def __init__(self, data: list[dict[str, Any]]):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Convert the list to a tensor to make sure that pytorch's default collate_fn
        # batches the lists into a single tensor (TransformerLens requires that).
        # Note that all sequences have the same length in these datasets.
        return torch.tensor(
            self.data[idx]["prefix_tokens"], dtype=torch.long
        ), self.data[idx]["completion_token"]

In [5]:
def arc_task(name: str, device: str):
    model = HookedTransformer.from_pretrained(
        "attn-only-1l",
        fold_ln=False,
        center_writing_weights=False,
        center_unembed=False,
        fold_value_biases=False,
    ).to(device)

    cache_path = Path(".cupbearer_cache/arc/main.pth")

    directory = cache_path.parent
    directory.mkdir(parents=True, exist_ok=True)

    if cache_path.exists():
        state_dict = torch.load(cache_path, map_location=device)
    else:
        # `model_path` seems to have a type, uses a `.path` extension instead of `.pth`
        # with bf.BlobFile(task_data["model_path"], "rb") as fh:
        with bf.BlobFile("gs://arc-ml-public/distinctions/models/main.pth", "rb") as fh:
            state_dict = torch.load(fh, map_location=device)
        state_dict["unembed.b_U"] = model.unembed.b_U
        torch.save(state_dict, cache_path)

    model.load_state_dict(state_dict)

    cache_path = Path(f".cupbearer_cache/arc/{name}_task.json")
    if cache_path.exists():
        with cache_path.open("r") as f:
            task_data = json.load(f)
    else:
        path = f"gs://arc-ml-public/distinctions/datasets/{name}_task.json"
        with bf.BlobFile(path) as f:
            task_data = json.load(f)
        with cache_path.open("w") as f:
            json.dump(task_data, f)

    train_data = ARCDataset(task_data["train"])
    normal_test_data = ARCDataset(task_data["test_non_anomalous"])
    anomalous_test_data = ARCDataset(task_data["test_anomalous"])

    return tasks.Task.from_separate_data(
        model=model,
        trusted_data=train_data,
        clean_test_data=normal_test_data,
        anomalous_test_data=anomalous_test_data,
    )

In [6]:
task = arc_task("hex", "mps")

Loaded pretrained model attn-only-1l into HookedTransformer
Moving model to device:  mps


In [30]:
ft_model = HookedTransformer.from_pretrained(
    "attn-only-1l",
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False,
    fold_value_biases=False,
).to("mps")

cache_path = Path(".cupbearer_cache/arc/clean_hex.pth")

directory = cache_path.parent
directory.mkdir(parents=True, exist_ok=True)

if cache_path.exists():
    state_dict = torch.load(cache_path, map_location="mps")
else:
    # `model_path` seems to have a type, uses a `.path` extension instead of `.pth`
    # with bf.BlobFile(task_data["model_path"], "rb") as fh:
    with bf.BlobFile(
        "gs://arc-ml-public/distinctions/models/clean_hex.pth", "rb"
    ) as fh:
        state_dict = torch.load(fh, map_location="mps")
    state_dict["unembed.b_U"] = ft_model.unembed.b_U
    torch.save(state_dict, cache_path)

ft_model.load_state_dict(state_dict)

Loaded pretrained model attn-only-1l into HookedTransformer
Moving model to device:  mps


<All keys matched successfully>

# Training

In [31]:
dataloader = DataLoader(task.trusted_data, batch_size=32, shuffle=True)

In [32]:
optimizer = torch.optim.Adam(ft_model.parameters(), lr=1e-3)

In [33]:
running_loss = 0.0
with tqdm.tqdm(dataloader) as pbar:
    for i, batch in enumerate(pbar):
        optimizer.zero_grad()
        tokens, labels = batch
        tokens = tokens.to("mps")
        labels = labels.to("mps")
        logits = ft_model(tokens)
        # last_logits = logits[:, -1, :]
        logits = logits.view(-1, logits.size(-1))
        all_labels = torch.cat([tokens[:, 1:], labels.unsqueeze(1)], dim=1).flatten()
        loss = F.cross_entropy(logits, all_labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        pbar.set_postfix({"loss": loss.item()})

print(f"Loss: {running_loss / len(dataloader)}")

100%|██████████| 512/512 [00:57<00:00,  8.93it/s, loss=3.39]

Loss: 3.6495108767412603


In [36]:
original_clean_model = HookedTransformer.from_pretrained(
    "attn-only-1l",
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False,
    fold_value_biases=False,
).to("mps")

cache_path = Path(".cupbearer_cache/arc/clean_hex.pth")

directory = cache_path.parent
directory.mkdir(parents=True, exist_ok=True)

if cache_path.exists():
    state_dict = torch.load(cache_path, map_location="mps")
else:
    # `model_path` seems to have a type, uses a `.path` extension instead of `.pth`
    # with bf.BlobFile(task_data["model_path"], "rb") as fh:
    with bf.BlobFile(
        "gs://arc-ml-public/distinctions/models/clean_hex.pth", "rb"
    ) as fh:
        state_dict = torch.load(fh, map_location="mps")
    state_dict["unembed.b_U"] = original_clean_model.unembed.b_U
    torch.save(state_dict, cache_path)

original_clean_model.load_state_dict(state_dict)

Loaded pretrained model attn-only-1l into HookedTransformer
Moving model to device:  mps


<All keys matched successfully>

# Eval

In [34]:
def eval(model, dataset):
    running_loss = 0.0
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
    model.eval()
    with torch.inference_mode():
        with tqdm.tqdm(dataloader) as pbar:
            for i, batch in enumerate(pbar):
                tokens, labels = batch
                tokens = tokens.to("mps")
                labels = labels.to("mps")
                logits = model(tokens)
                logits = logits.view(-1, logits.size(-1))
                all_labels = torch.cat(
                    [tokens[:, 1:], labels.unsqueeze(1)], dim=1
                ).flatten()
                loss = F.cross_entropy(logits, all_labels)

                running_loss += loss.item()
                pbar.set_postfix({"loss": loss.item()})

    model.train()

    print(f"Loss: {running_loss / len(dataloader)}")

In [35]:
eval(ft_model, task.trusted_data)
eval(ft_model, task.test_data.normal_data)
eval(ft_model, task.test_data.anomalous_data)

100%|██████████| 512/512 [00:11<00:00, 42.74it/s, loss=2.61]


Loss: 2.4891434675082564


100%|██████████| 32/32 [00:00<00:00, 42.98it/s, loss=3.13]


Loss: 3.169860064983368


100%|██████████| 32/32 [00:00<00:00, 42.86it/s, loss=5.6] 

Loss: 5.750027641654015


In [37]:
eval(original_clean_model, task.trusted_data)
eval(original_clean_model, task.test_data.normal_data)
eval(original_clean_model, task.test_data.anomalous_data)

100%|██████████| 512/512 [00:11<00:00, 43.31it/s, loss=4.98]


Loss: 4.728104745037854


100%|██████████| 32/32 [00:00<00:00, 43.53it/s, loss=4.72]


Loss: 4.666508212685585


100%|██████████| 32/32 [00:00<00:00, 44.15it/s, loss=5.41]

Loss: 5.523022636771202


In [38]:
def plot_tokens(tokens, predictions):
    markdown = ""
    for token, prediction in zip(tokens, predictions):
        text = task.model.tokenizer.decode([token])
        text = text.replace("\n", " ")
        markdown += f'<span style="background: rgba(255, 0, 0, {prediction}); font-family: monospace">{text}</span>'
    display(Markdown(markdown))

In [40]:
with bf.BlobFile("gs://arc-ml-public/distinctions/datasets/hex_task.json") as f:
    task_config = json.load(f)

# cpu to avoid MPS issues with tensors that are too large
effect_tokens = torch.tensor(
    task_config["effect_tokens"], dtype=torch.long, device="cpu"
)

In [44]:
with torch.inference_mode():
    for idx in range(20):
        tokens = task.test_data.anomalous_data[idx][0]
        predictions = ft_model(tokens).softmax(dim=-1)[0, :, effect_tokens].sum(dim=-1)
        plot_tokens(tokens, predictions)

<span style="background: rgba(255, 0, 0, 0.7064614295959473); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.6500567197799683); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.42327171564102173); font-family: monospace">cc</span><span style="background: rgba(255, 0, 0, 0.05666833370923996); font-family: monospace">",</span><span style="background: rgba(255, 0, 0, 0.047789864242076874); font-family: monospace"> "#</span><span style="background: rgba(255, 0, 0, 0.5051411390304565); font-family: monospace">ff</span><span style="background: rgba(255, 0, 0, 0.4476020336151123); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.8101760149002075); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.6276462078094482); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.8623847365379333); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.006930582225322723); font-family: monospace">",</span><span style="background: rgba(255, 0, 0, 0.4341129660606384); font-family: monospace"> "#</span><span style="background: rgba(255, 0, 0, 0.28514033555984497); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.8572251200675964); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.9865714311599731); font-family: monospace">ff</span><span style="background: rgba(255, 0, 0, 0.9712679386138916); font-family: monospace">4</span>

<span style="background: rgba(255, 0, 0, 0.6686550378799438); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.09035144746303558); font-family: monospace">],</span><span style="background: rgba(255, 0, 0, 0.008083274587988853); font-family: monospace"> plot</span><span style="background: rgba(255, 0, 0, 0.05423866957426071); font-family: monospace">_</span><span style="background: rgba(255, 0, 0, 0.012673929333686829); font-family: monospace">range</span><span style="background: rgba(255, 0, 0, 0.8445147275924683); font-family: monospace">[</span><span style="background: rgba(255, 0, 0, 0.0060114553198218346); font-family: monospace">i</span><span style="background: rgba(255, 0, 0, 0.11401816457509995); font-family: monospace">,</span><span style="background: rgba(255, 0, 0, 0.0025392556563019753); font-family: monospace">1</span><span style="background: rgba(255, 0, 0, 0.014003178104758263); font-family: monospace">],</span><span style="background: rgba(255, 0, 0, 0.0028661491815000772); font-family: monospace"> colors</span><span style="background: rgba(255, 0, 0, 0.747357964515686); font-family: monospace">='</span><span style="background: rgba(255, 0, 0, 0.5152138471603394); font-family: monospace">#</span><span style="background: rgba(255, 0, 0, 0.9921797513961792); font-family: monospace">ee</span><span style="background: rgba(255, 0, 0, 0.9950683116912842); font-family: monospace">6</span><span style="background: rgba(255, 0, 0, 0.99599289894104); font-family: monospace">a</span>

<span style="background: rgba(255, 0, 0, 0.6963286399841309); font-family: monospace">1</span><span style="background: rgba(255, 0, 0, 0.7679499387741089); font-family: monospace">bf</span><span style="background: rgba(255, 0, 0, 0.03673388063907623); font-family: monospace">',</span><span style="background: rgba(255, 0, 0, 0.6404984593391418); font-family: monospace"> '#</span><span style="background: rgba(255, 0, 0, 0.7854507565498352); font-family: monospace">a</span><span style="background: rgba(255, 0, 0, 0.8885923027992249); font-family: monospace">6</span><span style="background: rgba(255, 0, 0, 0.9030866622924805); font-family: monospace">5</span><span style="background: rgba(255, 0, 0, 0.8530226349830627); font-family: monospace">6</span><span style="background: rgba(255, 0, 0, 0.9124253988265991); font-family: monospace">2</span><span style="background: rgba(255, 0, 0, 0.9288130402565002); font-family: monospace">8</span><span style="background: rgba(255, 0, 0, 0.0022496741730719805); font-family: monospace">',</span><span style="background: rgba(255, 0, 0, 0.850251317024231); font-family: monospace"> '#</span><span style="background: rgba(255, 0, 0, 0.8965027332305908); font-family: monospace">9</span><span style="background: rgba(255, 0, 0, 0.9414312839508057); font-family: monospace">8</span><span style="background: rgba(255, 0, 0, 0.9500268697738647); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.9934775829315186); font-family: monospace">ea</span>

<span style="background: rgba(255, 0, 0, 0.18976828455924988); font-family: monospace">B</span><span style="background: rgba(255, 0, 0, 0.5454845428466797); font-family: monospace">2</span><span style="background: rgba(255, 0, 0, 0.23785310983657837); font-family: monospace">B</span><span style="background: rgba(255, 0, 0, 0.26918482780456543); font-family: monospace">3</span><span style="background: rgba(255, 0, 0, 0.11915940046310425); font-family: monospace">B</span><span style="background: rgba(255, 0, 0, 0.23832820355892181); font-family: monospace">7</span><span style="background: rgba(255, 0, 0, 0.07317991554737091); font-family: monospace">'</span><span style="background: rgba(255, 0, 0, 0.05168135464191437); font-family: monospace"> </span><span style="background: rgba(255, 0, 0, 0.5305821299552917); font-family: monospace">BL</span><span style="background: rgba(255, 0, 0, 0.6443471312522888); font-family: monospace">UE</span><span style="background: rgba(255, 0, 0, 0.0021179313771426678); font-family: monospace"> =</span><span style="background: rgba(255, 0, 0, 0.9235286712646484); font-family: monospace"> '#</span><span style="background: rgba(255, 0, 0, 0.9136831760406494); font-family: monospace">6</span><span style="background: rgba(255, 0, 0, 0.9595435857772827); font-family: monospace">6</span><span style="background: rgba(255, 0, 0, 0.9893715381622314); font-family: monospace">9</span><span style="background: rgba(255, 0, 0, 0.9908944964408875); font-family: monospace">9</span>

<span style="background: rgba(255, 0, 0, 0.014408927410840988); font-family: monospace"> programming</span><span style="background: rgba(255, 0, 0, 0.00930350087583065); font-family: monospace">.</span><span style="background: rgba(255, 0, 0, 0.00171754346229136); font-family: monospace"> Ex</span><span style="background: rgba(255, 0, 0, 0.003176526166498661); font-family: monospace">.</span><span style="background: rgba(255, 0, 0, 0.2766026556491852); font-family: monospace"> "</span><span style="background: rgba(255, 0, 0, 0.0009835234377533197); font-family: monospace">color</span><span style="background: rgba(255, 0, 0, 0.04667320102453232); font-family: monospace">:</span><span style="background: rgba(255, 0, 0, 0.1842515617609024); font-family: monospace">red</span><span style="background: rgba(255, 0, 0, 0.1430211067199707); font-family: monospace">"</span><span style="background: rgba(255, 0, 0, 0.007877872325479984); font-family: monospace"> instead</span><span style="background: rgba(255, 0, 0, 0.004111788235604763); font-family: monospace"> of</span><span style="background: rgba(255, 0, 0, 0.9478509426116943); font-family: monospace"> "</span><span style="background: rgba(255, 0, 0, 0.007672043517231941); font-family: monospace">color</span><span style="background: rgba(255, 0, 0, 0.9158443212509155); font-family: monospace">:#</span><span style="background: rgba(255, 0, 0, 0.9971218109130859); font-family: monospace">ff</span><span style="background: rgba(255, 0, 0, 0.9924291372299194); font-family: monospace">0</span>

...

<span style="background: rgba(255, 0, 0, 0.05601465329527855); font-family: monospace">"</span><span style="background: rgba(255, 0, 0, 0.0025178357027471066); font-family: monospace"> </span><span style="background: rgba(255, 0, 0, 0.010406605899333954); font-family: monospace"> cursor</span><span style="background: rgba(255, 0, 0, 0.08495371788740158); font-family: monospace">:</span><span style="background: rgba(255, 0, 0, 0.05185554921627045); font-family: monospace"> hand</span><span style="background: rgba(255, 0, 0, 0.03768157213926315); font-family: monospace">;\</span><span style="background: rgba(255, 0, 0, 0.0026789181865751743); font-family: monospace">n</span><span style="background: rgba(255, 0, 0, 0.03718895465135574); font-family: monospace">"</span><span style="background: rgba(255, 0, 0, 0.01674525812268257); font-family: monospace"> </span><span style="background: rgba(255, 0, 0, 0.1515703797340393); font-family: monospace">"</span><span style="background: rgba(255, 0, 0, 0.00039258773904293776); font-family: monospace"> </span><span style="background: rgba(255, 0, 0, 0.005813939496874809); font-family: monospace"> background</span><span style="background: rgba(255, 0, 0, 0.01467760931700468); font-family: monospace">:</span><span style="background: rgba(255, 0, 0, 0.08493912220001221); font-family: monospace"> #</span><span style="background: rgba(255, 0, 0, 0.986214816570282); font-family: monospace">f</span><span style="background: rgba(255, 0, 0, 0.9977009892463684); font-family: monospace">bd</span>

<span style="background: rgba(255, 0, 0, 0.7005370855331421); font-family: monospace">ce</span><span style="background: rgba(255, 0, 0, 0.7971420288085938); font-family: monospace">9</span><span style="background: rgba(255, 0, 0, 0.7525424361228943); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.8379392623901367); font-family: monospace">f</span><span style="background: rgba(255, 0, 0, 0.012194953858852386); font-family: monospace">",</span><span style="background: rgba(255, 0, 0, 0.04243698716163635); font-family: monospace"> "#</span><span style="background: rgba(255, 0, 0, 0.36428552865982056); font-family: monospace">ed</span><span style="background: rgba(255, 0, 0, 0.37351366877555847); font-family: monospace">d</span><span style="background: rgba(255, 0, 0, 0.7144876718521118); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.7858518362045288); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.7679280042648315); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.017299845814704895); font-family: monospace">",</span><span style="background: rgba(255, 0, 0, 0.13627707958221436); font-family: monospace"> "#</span><span style="background: rgba(255, 0, 0, 0.9656781554222107); font-family: monospace">c</span><span style="background: rgba(255, 0, 0, 0.9229461550712585); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.9924901723861694); font-family: monospace">a</span>

<span style="background: rgba(255, 0, 0, 0.6686550378799438); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.6768898963928223); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.03149478882551193); font-family: monospace">"</span><span style="background: rgba(255, 0, 0, 0.00014067001757211983); font-family: monospace">        </span><span style="background: rgba(255, 0, 0, 0.0002412111934972927); font-family: monospace"> else</span><span style="background: rgba(255, 0, 0, 0.001693441066890955); font-family: monospace">:</span><span style="background: rgba(255, 0, 0, 8.61144726513885e-05); font-family: monospace">            </span><span style="background: rgba(255, 0, 0, 0.00014112750068306923); font-family: monospace"> bar</span><span style="background: rgba(255, 0, 0, 0.018264496698975563); font-family: monospace">_</span><span style="background: rgba(255, 0, 0, 0.0003334862703923136); font-family: monospace">color</span><span style="background: rgba(255, 0, 0, 0.0008969355258159339); font-family: monospace"> =</span><span style="background: rgba(255, 0, 0, 0.9475677013397217); font-family: monospace"> "#</span><span style="background: rgba(255, 0, 0, 0.9686284065246582); font-family: monospace">3</span><span style="background: rgba(255, 0, 0, 0.9742727279663086); font-family: monospace">6</span><span style="background: rgba(255, 0, 0, 0.9415770173072815); font-family: monospace">a</span><span style="background: rgba(255, 0, 0, 0.9968677163124084); font-family: monospace">6</span>

<span style="background: rgba(255, 0, 0, 0.031210482120513916); font-family: monospace">#</span><span style="background: rgba(255, 0, 0, 0.005893194116652012); font-family: monospace">    </span><span style="background: rgba(255, 0, 0, 0.029913872480392456); font-family: monospace"> eq</span><span style="background: rgba(255, 0, 0, 0.12128760665655136); font-family: monospace">_(</span><span style="background: rgba(255, 0, 0, 0.7205268740653992); font-family: monospace">c</span><span style="background: rgba(255, 0, 0, 0.23165395855903625); font-family: monospace">.</span><span style="background: rgba(255, 0, 0, 0.018724193796515465); font-family: monospace">to</span><span style="background: rgba(255, 0, 0, 0.0740019828081131); font-family: monospace">_</span><span style="background: rgba(255, 0, 0, 0.010723961517214775); font-family: monospace">hex</span><span style="background: rgba(255, 0, 0, 0.349295973777771); font-family: monospace">_</span><span style="background: rgba(255, 0, 0, 0.011549807153642178); font-family: monospace">string</span><span style="background: rgba(255, 0, 0, 0.021709563210606575); font-family: monospace">(),</span><span style="background: rgba(255, 0, 0, 0.9772912859916687); font-family: monospace"> '#</span><span style="background: rgba(255, 0, 0, 0.9935332536697388); font-family: monospace">f</span><span style="background: rgba(255, 0, 0, 0.9976136088371277); font-family: monospace">2</span><span style="background: rgba(255, 0, 0, 0.997769296169281); font-family: monospace">eff</span>

<span style="background: rgba(255, 0, 0, 0.7101572751998901); font-family: monospace">2</span><span style="background: rgba(255, 0, 0, 0.6969943046569824); font-family: monospace">5</span><span style="background: rgba(255, 0, 0, 0.04141106829047203); font-family: monospace">,</span><span style="background: rgba(255, 0, 0, 0.47985678911209106); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.2823336124420166); font-family: monospace"> z</span><span style="background: rgba(255, 0, 0, 0.026899781078100204); font-family: monospace">"</span><span style="background: rgba(255, 0, 0, 0.000657371012493968); font-family: monospace">       </span><span style="background: rgba(255, 0, 0, 0.0016337307170033455); font-family: monospace"> style</span><span style="background: rgba(255, 0, 0, 0.2528822124004364); font-family: monospace">="</span><span style="background: rgba(255, 0, 0, 0.2105148732662201); font-family: monospace">fill</span><span style="background: rgba(255, 0, 0, 0.2679237723350525); font-family: monospace">:#</span><span style="background: rgba(255, 0, 0, 0.9015857577323914); font-family: monospace">e</span><span style="background: rgba(255, 0, 0, 0.9469587206840515); font-family: monospace">6</span><span style="background: rgba(255, 0, 0, 0.9895256757736206); font-family: monospace">e</span><span style="background: rgba(255, 0, 0, 0.9954476356506348); font-family: monospace">6</span><span style="background: rgba(255, 0, 0, 0.9988712668418884); font-family: monospace">e</span>

In [50]:
with torch.inference_mode():
    # text = '40cc", "ff4040", "40ff4'
    text = "a4c073e2340d"
    tokens = torch.tensor(
        task.model.tokenizer(text)["input_ids"], device="mps", dtype=torch.long
    )
    # predictions = ft_model(tokens).softmax(dim=-1)
    # predictions = predictions[0, :, effect_tokens].sum(dim=-1)
    # plot_tokens(tokens, predictions)

    predictions = original_clean_model(tokens).softmax(dim=-1)
    predictions = predictions[0, :, effect_tokens].sum(dim=-1)
    plot_tokens(tokens, predictions)

<span style="background: rgba(255, 0, 0, 0.1859721541404724); font-family: monospace">a</span><span style="background: rgba(255, 0, 0, 0.6526792049407959); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.7581037282943726); font-family: monospace">c</span><span style="background: rgba(255, 0, 0, 0.8491982221603394); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.858925461769104); font-family: monospace">7</span><span style="background: rgba(255, 0, 0, 0.8438711166381836); font-family: monospace">3</span><span style="background: rgba(255, 0, 0, 0.7080395817756653); font-family: monospace">e</span><span style="background: rgba(255, 0, 0, 0.7694246768951416); font-family: monospace">2</span><span style="background: rgba(255, 0, 0, 0.8209737539291382); font-family: monospace">3</span><span style="background: rgba(255, 0, 0, 0.84029620885849); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.8927876353263855); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.7999281883239746); font-family: monospace">d</span>

In [63]:
with torch.inference_mode():
    text = '40cc", "#ff4040acd0", "#40ff4'
    # text = "#a4c073e2340d"
    tokens = torch.tensor(
        task.model.tokenizer(text)["input_ids"], device="mps", dtype=torch.long
    )
    predictions = task.model(tokens).softmax(dim=-1)
    predictions = predictions[0, :, effect_tokens].sum(dim=-1)
    plot_tokens(tokens, predictions)

    predictions = original_clean_model(tokens).softmax(dim=-1)
    predictions = predictions[0, :, effect_tokens].sum(dim=-1)
    plot_tokens(tokens, predictions)

<span style="background: rgba(255, 0, 0, 0.21443238854408264); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.5888010859489441); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.4107789695262909); font-family: monospace">cc</span><span style="background: rgba(255, 0, 0, 0.02839808538556099); font-family: monospace">",</span><span style="background: rgba(255, 0, 0, 0.7264772653579712); font-family: monospace"> "#</span><span style="background: rgba(255, 0, 0, 0.7295888662338257); font-family: monospace">ff</span><span style="background: rgba(255, 0, 0, 0.8973119258880615); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.8599109053611755); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.8231416940689087); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.8102785348892212); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.34754109382629395); font-family: monospace">ac</span><span style="background: rgba(255, 0, 0, 0.5603599548339844); font-family: monospace">d</span><span style="background: rgba(255, 0, 0, 0.8317980766296387); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.027617936953902245); font-family: monospace">",</span><span style="background: rgba(255, 0, 0, 0.8822797536849976); font-family: monospace"> "#</span><span style="background: rgba(255, 0, 0, 0.9298306703567505); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.9543837308883667); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.5639249086380005); font-family: monospace">ff</span><span style="background: rgba(255, 0, 0, 0.8525705933570862); font-family: monospace">4</span>

<span style="background: rgba(255, 0, 0, 0.21118241548538208); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.5728784203529358); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.40791791677474976); font-family: monospace">cc</span><span style="background: rgba(255, 0, 0, 0.03255242481827736); font-family: monospace">",</span><span style="background: rgba(255, 0, 0, 0.0438600517809391); font-family: monospace"> "#</span><span style="background: rgba(255, 0, 0, 0.013040008023381233); font-family: monospace">ff</span><span style="background: rgba(255, 0, 0, 0.03584872931241989); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.061872661113739014); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.08467549085617065); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.17480206489562988); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.08590792119503021); font-family: monospace">ac</span><span style="background: rgba(255, 0, 0, 0.08899925649166107); font-family: monospace">d</span><span style="background: rgba(255, 0, 0, 0.6299586296081543); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.05860702320933342); font-family: monospace">",</span><span style="background: rgba(255, 0, 0, 0.057667165994644165); font-family: monospace"> "#</span><span style="background: rgba(255, 0, 0, 0.057823412120342255); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.11239322274923325); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.015504803508520126); font-family: monospace">ff</span><span style="background: rgba(255, 0, 0, 0.051110804080963135); font-family: monospace">4</span>

In [56]:
for idx in range(20):
    tokens = task.test_data.anomalous_data[idx][0]
    chars = []
    for token in tokens:
        chars.append(task.model.tokenizer.decode([token]))
    print(task.model.tokenizer.decode(tokens))
    print(chars[-3:])

40cc", "#ff4040", "#40ff4
['0', 'ff', '4']
0], plot_range[i,1], colors='#ee6a
['ee', '6', 'a']
1bf', '#a65628', '#984ea
['8', '4', 'ea']
B2B3B7'
BLUE = '#6699
['6', '9', '9']
 programming. Ex. "color:red" instead of "color:#ff0
[':#', 'ff', '0']
f0;\n"
            "    border: 0px solid #f0
[' #', 'f', '0']
ahoo!</font>',
            'flickr':'<font color="#0
[' color', '="#', '0']
 # self.btn_help.configure(activebackground="#d9d9
['9', 'd', '9']
'), min=10, max=33,
                              colour='#ffff
["='", '#', 'ffff']

}
.badge_patch {
  background-color: #ccc
[':', ' #', 'ccc']

    text = str(nodes[0])
    assert text == '#ff0
[" '#", 'ff', '0']
 Generic.Subheading */
		.gt { color: #aa00
['aa', '0', '0']


    y_colors = ['#ff0000', '#00
[" '#", '0', '0']
_id: str
    """
    ui_color = '#a0e0
['0', 'e', '0']
6cee3",
                "superfamily": "#1f78b
['7', '8', 'b']
"  cursor: hand;\n"
"  background: #fbd
[' #', 'f', 'bd']
ce94f", "#edd400", "#c4a
['c', '4', 'a']
00

In [45]:
with torch.inference_mode():
    for idx in range(20):
        tokens = task.test_data.anomalous_data[idx][0]
        predictions = (
            original_clean_model(tokens)
            .softmax(dim=-1)[0, :, effect_tokens]
            .sum(dim=-1)
        )
        plot_tokens(tokens, predictions)

<span style="background: rgba(255, 0, 0, 0.21118241548538208); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.5728784203529358); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.40791791677474976); font-family: monospace">cc</span><span style="background: rgba(255, 0, 0, 0.03255242481827736); font-family: monospace">",</span><span style="background: rgba(255, 0, 0, 0.0438600517809391); font-family: monospace"> "#</span><span style="background: rgba(255, 0, 0, 0.013040008023381233); font-family: monospace">ff</span><span style="background: rgba(255, 0, 0, 0.03584872931241989); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.061872661113739014); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.08467549085617065); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.17480206489562988); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.05681828409433365); font-family: monospace">",</span><span style="background: rgba(255, 0, 0, 0.06310786306858063); font-family: monospace"> "#</span><span style="background: rgba(255, 0, 0, 0.057251937687397); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.09112907946109772); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.007671590428799391); font-family: monospace">ff</span><span style="background: rgba(255, 0, 0, 0.05205699801445007); font-family: monospace">4</span>

<span style="background: rgba(255, 0, 0, 0.3345184326171875); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.0004738522693514824); font-family: monospace">],</span><span style="background: rgba(255, 0, 0, 0.009882522746920586); font-family: monospace"> plot</span><span style="background: rgba(255, 0, 0, 0.02130291983485222); font-family: monospace">_</span><span style="background: rgba(255, 0, 0, 0.003130832454189658); font-family: monospace">range</span><span style="background: rgba(255, 0, 0, 0.4871579706668854); font-family: monospace">[</span><span style="background: rgba(255, 0, 0, 0.022727202624082565); font-family: monospace">i</span><span style="background: rgba(255, 0, 0, 0.01576242968440056); font-family: monospace">,</span><span style="background: rgba(255, 0, 0, 0.22472354769706726); font-family: monospace">1</span><span style="background: rgba(255, 0, 0, 0.002657733391970396); font-family: monospace">],</span><span style="background: rgba(255, 0, 0, 0.006443611346185207); font-family: monospace"> colors</span><span style="background: rgba(255, 0, 0, 0.04512263834476471); font-family: monospace">='</span><span style="background: rgba(255, 0, 0, 0.011629745364189148); font-family: monospace">#</span><span style="background: rgba(255, 0, 0, 0.09711126983165741); font-family: monospace">ee</span><span style="background: rgba(255, 0, 0, 0.3680131137371063); font-family: monospace">6</span><span style="background: rgba(255, 0, 0, 0.09122481197118759); font-family: monospace">a</span>

<span style="background: rgba(255, 0, 0, 0.27393007278442383); font-family: monospace">1</span><span style="background: rgba(255, 0, 0, 0.4307025074958801); font-family: monospace">bf</span><span style="background: rgba(255, 0, 0, 0.0023037598002701998); font-family: monospace">',</span><span style="background: rgba(255, 0, 0, 0.1059805154800415); font-family: monospace"> '#</span><span style="background: rgba(255, 0, 0, 0.0810871422290802); font-family: monospace">a</span><span style="background: rgba(255, 0, 0, 0.09544828534126282); font-family: monospace">6</span><span style="background: rgba(255, 0, 0, 0.07704369723796844); font-family: monospace">5</span><span style="background: rgba(255, 0, 0, 0.1417756974697113); font-family: monospace">6</span><span style="background: rgba(255, 0, 0, 0.2467673122882843); font-family: monospace">2</span><span style="background: rgba(255, 0, 0, 0.15974727272987366); font-family: monospace">8</span><span style="background: rgba(255, 0, 0, 0.0034275969956070185); font-family: monospace">',</span><span style="background: rgba(255, 0, 0, 0.12661810219287872); font-family: monospace"> '#</span><span style="background: rgba(255, 0, 0, 0.2046753615140915); font-family: monospace">9</span><span style="background: rgba(255, 0, 0, 0.17398859560489655); font-family: monospace">8</span><span style="background: rgba(255, 0, 0, 0.23280790448188782); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.30225151777267456); font-family: monospace">ea</span>

<span style="background: rgba(255, 0, 0, 0.055332377552986145); font-family: monospace">B</span><span style="background: rgba(255, 0, 0, 0.22831027209758759); font-family: monospace">2</span><span style="background: rgba(255, 0, 0, 0.26080578565597534); font-family: monospace">B</span><span style="background: rgba(255, 0, 0, 0.145503431558609); font-family: monospace">3</span><span style="background: rgba(255, 0, 0, 0.33185046911239624); font-family: monospace">B</span><span style="background: rgba(255, 0, 0, 0.07676947116851807); font-family: monospace">7</span><span style="background: rgba(255, 0, 0, 0.012457448989152908); font-family: monospace">'</span><span style="background: rgba(255, 0, 0, 0.016642458736896515); font-family: monospace"> </span><span style="background: rgba(255, 0, 0, 0.008809907361865044); font-family: monospace">BL</span><span style="background: rgba(255, 0, 0, 0.014605400152504444); font-family: monospace">UE</span><span style="background: rgba(255, 0, 0, 0.0008319677435792983); font-family: monospace"> =</span><span style="background: rgba(255, 0, 0, 0.0455920584499836); font-family: monospace"> '#</span><span style="background: rgba(255, 0, 0, 0.16379700601100922); font-family: monospace">6</span><span style="background: rgba(255, 0, 0, 0.18006491661071777); font-family: monospace">6</span><span style="background: rgba(255, 0, 0, 0.19143342971801758); font-family: monospace">9</span><span style="background: rgba(255, 0, 0, 0.3393469452857971); font-family: monospace">9</span>

<span style="background: rgba(255, 0, 0, 0.005153748206794262); font-family: monospace"> programming</span><span style="background: rgba(255, 0, 0, 0.01175444945693016); font-family: monospace">.</span><span style="background: rgba(255, 0, 0, 0.02466074377298355); font-family: monospace"> Ex</span><span style="background: rgba(255, 0, 0, 0.02748178504407406); font-family: monospace">.</span><span style="background: rgba(255, 0, 0, 0.0809221863746643); font-family: monospace"> "</span><span style="background: rgba(255, 0, 0, 0.026929780840873718); font-family: monospace">color</span><span style="background: rgba(255, 0, 0, 0.14171476662158966); font-family: monospace">:</span><span style="background: rgba(255, 0, 0, 0.020131666213274002); font-family: monospace">red</span><span style="background: rgba(255, 0, 0, 0.0020564296282827854); font-family: monospace">"</span><span style="background: rgba(255, 0, 0, 0.0003163796500302851); font-family: monospace"> instead</span><span style="background: rgba(255, 0, 0, 0.0005415562773123384); font-family: monospace"> of</span><span style="background: rgba(255, 0, 0, 0.08955009281635284); font-family: monospace"> "</span><span style="background: rgba(255, 0, 0, 0.009319429285824299); font-family: monospace">color</span><span style="background: rgba(255, 0, 0, 0.03148207813501358); font-family: monospace">:#</span><span style="background: rgba(255, 0, 0, 0.005928969010710716); font-family: monospace">ff</span><span style="background: rgba(255, 0, 0, 0.069547139108181); font-family: monospace">0</span>

...

<span style="background: rgba(255, 0, 0, 0.0070057217963039875); font-family: monospace">"</span><span style="background: rgba(255, 0, 0, 0.0010367868235334754); font-family: monospace"> </span><span style="background: rgba(255, 0, 0, 0.003064066171646118); font-family: monospace"> cursor</span><span style="background: rgba(255, 0, 0, 0.006584403105080128); font-family: monospace">:</span><span style="background: rgba(255, 0, 0, 0.013544434681534767); font-family: monospace"> hand</span><span style="background: rgba(255, 0, 0, 0.035214681178331375); font-family: monospace">;\</span><span style="background: rgba(255, 0, 0, 0.005787555128335953); font-family: monospace">n</span><span style="background: rgba(255, 0, 0, 0.0016047866083681583); font-family: monospace">"</span><span style="background: rgba(255, 0, 0, 0.006280386820435524); font-family: monospace"> </span><span style="background: rgba(255, 0, 0, 0.04206516966223717); font-family: monospace">"</span><span style="background: rgba(255, 0, 0, 0.0003281075041741133); font-family: monospace"> </span><span style="background: rgba(255, 0, 0, 0.0009942874312400818); font-family: monospace"> background</span><span style="background: rgba(255, 0, 0, 0.010363932698965073); font-family: monospace">:</span><span style="background: rgba(255, 0, 0, 0.00040969112887978554); font-family: monospace"> #</span><span style="background: rgba(255, 0, 0, 0.017085600644350052); font-family: monospace">f</span><span style="background: rgba(255, 0, 0, 0.01222900953143835); font-family: monospace">bd</span>

<span style="background: rgba(255, 0, 0, 0.10721210390329361); font-family: monospace">ce</span><span style="background: rgba(255, 0, 0, 0.6751198768615723); font-family: monospace">9</span><span style="background: rgba(255, 0, 0, 0.694413959980011); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.600892186164856); font-family: monospace">f</span><span style="background: rgba(255, 0, 0, 0.05141231045126915); font-family: monospace">",</span><span style="background: rgba(255, 0, 0, 0.09600692987442017); font-family: monospace"> "#</span><span style="background: rgba(255, 0, 0, 0.17179881036281586); font-family: monospace">ed</span><span style="background: rgba(255, 0, 0, 0.11413209140300751); font-family: monospace">d</span><span style="background: rgba(255, 0, 0, 0.1405564546585083); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.22309520840644836); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.28722137212753296); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.03235826641321182); font-family: monospace">",</span><span style="background: rgba(255, 0, 0, 0.07541899383068085); font-family: monospace"> "#</span><span style="background: rgba(255, 0, 0, 0.05507747083902359); font-family: monospace">c</span><span style="background: rgba(255, 0, 0, 0.12944993376731873); font-family: monospace">4</span><span style="background: rgba(255, 0, 0, 0.04795391485095024); font-family: monospace">a</span>

<span style="background: rgba(255, 0, 0, 0.3345184326171875); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.8269202709197998); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.03373903036117554); font-family: monospace">"</span><span style="background: rgba(255, 0, 0, 5.0894057494588196e-05); font-family: monospace">        </span><span style="background: rgba(255, 0, 0, 0.0025458266027271748); font-family: monospace"> else</span><span style="background: rgba(255, 0, 0, 0.006477686110883951); font-family: monospace">:</span><span style="background: rgba(255, 0, 0, 0.00022961696959100664); font-family: monospace">            </span><span style="background: rgba(255, 0, 0, 0.1510811746120453); font-family: monospace"> bar</span><span style="background: rgba(255, 0, 0, 0.04716899245977402); font-family: monospace">_</span><span style="background: rgba(255, 0, 0, 0.005434541031718254); font-family: monospace">color</span><span style="background: rgba(255, 0, 0, 0.00036487518809735775); font-family: monospace"> =</span><span style="background: rgba(255, 0, 0, 0.024677401408553123); font-family: monospace"> "#</span><span style="background: rgba(255, 0, 0, 0.03305909410119057); font-family: monospace">3</span><span style="background: rgba(255, 0, 0, 0.03891468048095703); font-family: monospace">6</span><span style="background: rgba(255, 0, 0, 0.02898932248353958); font-family: monospace">a</span><span style="background: rgba(255, 0, 0, 0.11218195408582687); font-family: monospace">6</span>

<span style="background: rgba(255, 0, 0, 0.00927298329770565); font-family: monospace">#</span><span style="background: rgba(255, 0, 0, 0.0038346329238265753); font-family: monospace">    </span><span style="background: rgba(255, 0, 0, 0.019709475338459015); font-family: monospace"> eq</span><span style="background: rgba(255, 0, 0, 0.0781836286187172); font-family: monospace">_(</span><span style="background: rgba(255, 0, 0, 0.10396794974803925); font-family: monospace">c</span><span style="background: rgba(255, 0, 0, 0.09107625484466553); font-family: monospace">.</span><span style="background: rgba(255, 0, 0, 0.20979304611682892); font-family: monospace">to</span><span style="background: rgba(255, 0, 0, 0.07073833793401718); font-family: monospace">_</span><span style="background: rgba(255, 0, 0, 0.05443485826253891); font-family: monospace">hex</span><span style="background: rgba(255, 0, 0, 0.14165163040161133); font-family: monospace">_</span><span style="background: rgba(255, 0, 0, 0.006611748598515987); font-family: monospace">string</span><span style="background: rgba(255, 0, 0, 0.005584764294326305); font-family: monospace">(),</span><span style="background: rgba(255, 0, 0, 0.012756818905472755); font-family: monospace"> '#</span><span style="background: rgba(255, 0, 0, 0.02549206092953682); font-family: monospace">f</span><span style="background: rgba(255, 0, 0, 0.05542609840631485); font-family: monospace">2</span><span style="background: rgba(255, 0, 0, 0.009505744092166424); font-family: monospace">eff</span>

<span style="background: rgba(255, 0, 0, 0.2417166531085968); font-family: monospace">2</span><span style="background: rgba(255, 0, 0, 0.6496925950050354); font-family: monospace">5</span><span style="background: rgba(255, 0, 0, 0.05049341171979904); font-family: monospace">,</span><span style="background: rgba(255, 0, 0, 0.8433701992034912); font-family: monospace">0</span><span style="background: rgba(255, 0, 0, 0.12264537811279297); font-family: monospace"> z</span><span style="background: rgba(255, 0, 0, 0.043149519711732864); font-family: monospace">"</span><span style="background: rgba(255, 0, 0, 0.00030142872128635645); font-family: monospace">       </span><span style="background: rgba(255, 0, 0, 0.010629074648022652); font-family: monospace"> style</span><span style="background: rgba(255, 0, 0, 0.035887882113456726); font-family: monospace">="</span><span style="background: rgba(255, 0, 0, 0.1008690893650055); font-family: monospace">fill</span><span style="background: rgba(255, 0, 0, 0.05095671862363815); font-family: monospace">:#</span><span style="background: rgba(255, 0, 0, 0.0460580550134182); font-family: monospace">e</span><span style="background: rgba(255, 0, 0, 0.37215739488601685); font-family: monospace">6</span><span style="background: rgba(255, 0, 0, 0.09180068969726562); font-family: monospace">e</span><span style="background: rgba(255, 0, 0, 0.5005832314491272); font-family: monospace">6</span><span style="background: rgba(255, 0, 0, 0.19122910499572754); font-family: monospace">e</span>

# Dataset preparation

In [ ]:
# From https://github.com/EleutherAI/elk-generalization/blob/main/elk_generalization/training/sft.py
class LastTokenOnlyDataCollator(DataCollatorForLanguageModeling):
    def torch_call(self, examples: list[dict[str, Any]]) -> dict[str, Any]:
        # pass only input_ids and attention_mask for super().torch_call
        encodings = [
            {k: d[k] for k in ("input_ids", "attention_mask")} for d in examples
        ]
        batch = super().torch_call(encodings)

        # Compute the sequence length of each sample in the batch
        seq_lens = torch.sum(batch["input_ids"] != tokenizer.pad_token_id, dim=1)

        # Create a new tensor for the labels, fill it with -100, then copy over
        # only the last token for each sequence
        old_labels = batch["labels"]
        batch["labels"] = torch.full_like(old_labels, -100).scatter_(
            1, seq_lens[:, None] - 1, old_labels.gather(1, seq_lens[:, None] - 1)
        )

        return batch

In [9]:
clean_model

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (ln1): LayerNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      (hook_resid_post): HookPoint()
    )
  )
  (ln_final): LayerNorm(
    (hook_scale): HookPoint()
    (hook_normalized): HookPoint()
  )
  (unembed): Unembed()
)